In [1]:
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat

In [2]:
val spark = SparkSession.builder.master("local[*]").appName("SparkSQL").getOrCreate()
import spark.implicits._

// Reading the file from disk
val conf = new Configuration
conf.set("textinputformat.record.delimiter", "WARC/1.0")
val dataset = spark.sparkContext.newAPIHadoopFile("file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/CC-MAIN-20180918130631-20180918150631-00000.warc.wet", classOf[TextInputFormat], classOf[LongWritable], classOf[Text], conf)
val tempdata = dataset.map(x=>x._2.toString)      

val data = tempdata.mapPartitionsWithIndex {  // Drop invalid records
                    (idx, iter) => if (idx == 0) iter.drop(2) else iter 
                  }

// data.take(1).foreach(println)

spark = org.apache.spark.sql.SparkSession@749cc51f
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
dataset = file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/CC-MAIN-20180918130631-20180918150631-00000.warc.wet NewHadoopRDD[0] at newAPIHadoopFile at <console>:36
tempdata = MapPartitionsRDD[1] at map at <console>:37
data = MapPartitionsRDD[2] at mapPartitionsWithIndex at <console>:39


MapPartitionsRDD[2] at mapPartitionsWithIndex at <console>:39

In [36]:
data.take(3).foreach(println)                        


WARC-Type: conversion
WARC-Target-URI: http://0-50.ru/news/tag/%F4%F3%F2%E1%EE%EB+%D0%EE%F1%F1%E8%E8?page=80
WARC-Date: 2018-09-18T13:25:39Z
WARC-Record-ID: <urn:uuid:1ed56066-208a-4bdb-8b3c-a2ffc35b9802>
WARC-Refers-To: <urn:uuid:f913b540-0874-44a0-ae86-3549ae53096b>
WARC-Block-Digest: sha1:NUM333XMZOLA3EUJOKUKBG7ALV4G3K7F
Content-Type: text/plain
Content-Length: 17302

Новости 0-50.ru | Новости по тегу: футбол России
|
Погода в Екатеринбурге и Свердловской области |
Что приготовить на ужин рецепт с фото |
Новости Екатеринбурга |
96women.ru - Живой Женский Журнал |
Главная
Редакция
Реклама
О проекте
RSS
Обратная связь
Все новости
Екатеринбург
Россия и мир
Образование
Недвижимость
Здоровье
Спорт
Происшествия
Транспорт
Новости компаний
Другая жизнь
Статьи
Новости Екатеринбурга
В Екатеринбурге на Московской водитель Chevrolet Cruze протаранил дерево и сбежал в лес
В Екатеринбурге во дворе дома №5 в переулке Красном сожгли Toyota и повредили Hyundai Solaris
Под Богдановичем девушка на ВА

In [3]:
// List of tuples containing (metadata, file_content)
import java.net.URL
// import sqlContext.implicits._

val uriDataPairs = data.map(record => (record.split("\r\n\r\n")))
                        .map(dataItemAsList => {
                            // Paris of (Property, Value) from the meta data
                            val keyValuePairs = dataItemAsList(0).split("\r\n").filter(_ != "").map(line => {
                                val splitList = line.split(": ")
                                ((splitList(0), splitList(1)))
                            }) 
                            
                            // Selecting the first (and only) record and the value from that
                            val targetURL = keyValuePairs.filter(_._1 == "WARC-Target-URI")(0)._2
                            
                            ((targetURL,dataItemAsList(1)))
                        })//.take(2)(1)._1

// Create DF
val pageInfo = uriDataPairs.map(item => {
    val host = new URL(item._1).getHost
    val domainParts = host.replaceAll("""([^\.]*+\.)""", "")//.split(".") //.map(arr => arr.last
    (item._1, domainParts, item._2) // (url.getHost, url.getPath,
})//.take(1)

// val topDomainCom = pageInfo.filter(x)

val pageInfoDF = pageInfo.toDF("uri", "topDomain", "text")
pageInfoDF.show

+--------------------+---------+--------------------+
|                 uri|topDomain|                text|
+--------------------+---------+--------------------+
|http://0-50.ru/ne...|       ru|Новости 0-50.ru |...|
|http://000elham00...|      com|تبلیغات	
مسخره - ...|
|http://0105335908...|       kr|맛집? 많은분 들이 이곳을 이용...|
|http://01ninchisy...|      net|認知症ケアマニュアル: アルツハイ...|
|http://01xxxx.com...|      com|正在播放 5-鬼畜老师强暴学生妹[...|
|http://023pifu.co...|      com|皮肤病 / 荨麻疹_重庆皮肤病医院...|
|http://03e98e7.ne...|      com|Amicus Brief FAQs...|
|http://0411auto.c...|      com|2017款宝马X5 xDrive3...|
|http://0412hitomi...|      com|こら！ぼたち！ ダッシュするときも...|
|http://080ek21.co...|      com|網路聊天聯盟
回 首 頁 │ 點數...|
|http://09irk.ru/r...|       ru|Мангал | Леко мно...|
|http://0ino0.beon...|       ru|Sasori в дневнике...|
|http://0we.kunlun...|      com|Welcome 0we.kunlu...|
|http://1-800-opti...|      com|Unisex Patches
1-...|
|http://1-tech.com...|       ua|Купить CABHD-360-...|
|http://10-carat-d...|      

uriDataPairs = MapPartitionsRDD[4] at map at <console>:42
pageInfo = MapPartitionsRDD[5] at map at <console>:56
pageInfoDF = [uri: string, topDomain: string ... 1 more field]


[uri: string, topDomain: string ... 1 more field]

In [262]:
pageInfoDF.select("text").show

+--------------------+
|                text|
+--------------------+
|Новости 0-50.ru |...|
|تبلیغات	
مسخره - ...|
|맛집? 많은분 들이 이곳을 이용...|
|認知症ケアマニュアル: アルツハイ...|
|正在播放 5-鬼畜老师强暴学生妹[...|
|皮肤病 / 荨麻疹_重庆皮肤病医院...|
|Amicus Brief FAQs...|
|2017款宝马X5 xDrive3...|
|こら！ぼたち！ ダッシュするときも...|
|網路聊天聯盟
回 首 頁 │ 點數...|
|Мангал | Леко мно...|
|Sasori в дневнике...|
|Welcome 0we.kunlu...|
|Unisex Patches
1-...|
|Купить CABHD-360-...|
|Fascination About...|
|#19 Remembering a...|
|- На какой глубин...|
|Пораненого в Крам...|
|🎼 Слушать Lost i...|
+--------------------+
only showing top 20 rows



In [7]:
// val stopWords = Set("i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now")

// val keyWords = expandedURLDataPairs.map( item => {
//     // Extract keywords
//     val keywords = item._3.split("[\\s|\n]")
//                           .map(s => s.replaceAll("""[^\x00-\x7F]""", "")) // We only manage ascii chars
//                           .map(s => s.replaceAll("""^\p{Punct}*$""", "")) // No words with only punctuation
//                           .map(s => s.replaceAll("""\p{Punct}$""", ""))   // No trailing punctuations
//                           .filter(_ != "")                                // No empty words
//                           .map(_.toLowerCase)
//                           .filter(!stopWords.contains(_))                 // Remove stop-words
//                           .groupBy(identity).mapValues(_.size).toArray
//                           .sortBy(- _._2)

    
//     ((item._1, item._2, keywords))
// })

// val keyWordsDF = pageInfoDF.select("text").map(item => {
//     // Extract keywords
//     val text = item.getAs[String]("text") 
    
//     val keywords = text
//               .split("[\\s|\n]")
//               .map(s => s.replaceAll("""[^\x00-\x7F]""", "")) // We only manage ascii chars
//               .map(s => s.replaceAll("""^\p{Punct}*$""", "")) // No words with only punctuation
//               .map(s => s.replaceAll("""\p{Punct}$""", ""))   // No trailing punctuations
// //               .filter(_ != "")                                // No empty words
// //               .map(_.toLowerCase)
// //               .filter(!stopWords.contains(_))                 // Remove stop-words
// //               .groupBy(identity).mapValues(_.size).toArray
// //               .sortBy(- _._2)
//     (keywords)
// })
import org.apache.spark.sql.functions._ 

val extractKeyWords = udf[Array[(String, Int)], String](input => {
    val stopWords = Set("i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now")
    
    input
      .split("[\\s|\n]")
      .map(s => s.replaceAll("""[^\x00-\x7F]""", "")) // We only manage ascii chars
      .map(s => s.replaceAll("""^\p{Punct}*$""", "")) // No words with only punctuation
      .map(s => s.replaceAll("""\p{Punct}$""", ""))   // No trailing punctuations
      .filter(_ != "")                                // No empty words
      .map(_.toLowerCase)
      .filter(!stopWords.contains(_))                 // Remove stop-words
      .groupBy(identity).mapValues(_.size).toArray
      .sortBy(- _._2)
})


val keywordDF = pageInfoDF.withColumn(
    "keywords",
    extractKeyWords(col("text"))
  )

keywordDF
    .select("keywords")
    .show()

// keyWords.take(10).foreach(_._3.foreach(println))
// keyWordsDF.show


+--------------------+
|            keywords|
+--------------------+
|[[2, 13], [1:1, 4...|
|[[1396, 15], [139...|
|[[01053359081.kte...|
|[[(7, 2], [(5, 2]...|
|[[2017-02-16, 13]...|
|[[hospital, 3], [...|
|[[appellate, 4], ...|
|[[xdrive35i, 3], ...|
|[[(31, 15], [(30,...|
|[[ut, 40], [0401,...|
|[[+7(3952, 3], [p...|
|[[re, 7], [needle...|
|[[us, 2], [web, 2...|
|[[lens, 37], [pat...|
|[[hdmi, 43], [dvi...|
|[[dating, 7], [ad...|
|[[first, 8], [awe...|
|[[0,03, 2], [1800...|
|[[2015, 18], [201...|
|[[linkin, 24], [p...|
+--------------------+
only showing top 20 rows



extractKeyWords = UserDefinedFunction(<function1>,ArrayType(StructType(StructField(_1,StringType,true), StructField(_2,IntegerType,false)),true),Some(List(StringType)))
keywordDF = [uri: string, topDomain: string ... 2 more fields]


[uri: string, topDomain: string ... 2 more fields]

In [36]:
// create one list with all keywords
// keywords distinct = 2017741
// keywords total = 15 000 000
// total_nr_records = 39653
val keywords = keywordDF.withColumn("keywords", explode($"keywords._1")).select("keywords")//.distinct.count
// keywords.rdd.map(_.toString).saveAsTextFile("file:///home/hrabo/Documents/skola/data-intensive-computing/project/data/keywords.txt")

keywords = [keywords: string]


lastException: Throwable = null


[keywords: string]

In [ ]:
// Saving to cassandra

// connect to Cassandra and make a keyspace and table as explained in the document
val cluster = Cluster.builder().addContactPoint("127.0.0.1").build()
val session = cluster.connect()

session.execute("CREATE KEYSPACE IF NOT EXISTS search WITH REPLICATION =" +
                "{'class': 'SimpleStrategy', 'replication_factor': 1};")
session.execute("CREATE TABLE IF NOT EXISTS search.keywords (word text PRIMARY KEY, count float);");


keywordDF.write.format("org.apache.spark.sql.cassandra")
.options(Map("keyspace"->"search","table"->"keywords"))
.mode(SaveMode.Append)
.save()